In [47]:
import numpy as np
from collections import deque
import random

def compute_potentials(basis, C, m, n):
    """
    Compute the potential values for each point.
    :param basis: The set of current basis variables {(i, j)}, representing the spanning tree.
    :param C: Cost matrix.
    :param m: Number of supply points.
    :param n: Number of demand points.
    :return: Supply point potential, demand point potential.
    """
    potentials = np.full(m + n, None)  # Initialize all potentials, None indicates unassigned values
    
    # Select a starting point (usually supply point 0)
    start_node = 0
    potentials[start_node] = 0 

    # Construct an adjacency list (Graph) to represent the spanning tree
    tree = {i: [] for i in range(m + n)}
    for i, j in basis:
        tree[i].append(j + m)  
        tree[j + m].append(i)  

    # Use BFS to traverse the spanning tree and compute potential values
    queue = deque([start_node])
    
    while queue:
        current = queue.popleft()
        for neighbor in tree[current]:
            if potentials[neighbor] is None:  # Only compute unassigned points
                if current < m:  # Current node is a supply point
                    cost = C[current, neighbor - m]  # Demand point index needs offset adjustment by m
                    potentials[neighbor] = potentials[current] - cost
                else:  # Current node is a demand point
                    cost = C[neighbor, current - m]  # The supply point index is `neighbor`
                    potentials[neighbor] = potentials[current] + cost
                queue.append(neighbor)

    return potentials[:m], potentials[m:]  




In [48]:
import numpy as np


def find_min_reduced_cost(C, p_supply, p_demand, basis, y):
    """
    计算 y 集合中变量的 reduced cost，并找到最小的变量作为进入基底的变量
    :param C: 成本矩阵
    :param p_supply: 供应点的 potential 值
    :param p_demand: 需求点的 potential 值
    :param basis: 当前基底变量集合 {(i, j)}
    :param y: 目标变量集合 {(i, j)}
    :return: 进入基底的变量 (i, j)，若无可选变量返回 None
    """
    min_value = float('inf')
    entering_variable = None

    for (i, j) in y:
        if (i, j) not in basis:
            reduced_cost = C[i, j] - (p_supply[i] - p_demand[j])

            if reduced_cost < min_value:
                min_value = reduced_cost
                entering_variable = (i, j)

    # 若最小 reduced cost >= 0，则返回 None，表示最优解
    return entering_variable if min_value < 0 else None


def check_reduced_costs(C, p_supply, p_demand, basis):
    """
    计算所有变量的 reduced cost 并检查是否满足最优条件。
    :param C: 成本矩阵
    :param p_supply: 供应点的 potential 值
    :param p_demand: 需求点的 potential 值
    :param basis: 当前基底变量集合
    :return: bool (是否满足最优条件)
    """
    m, n = C.shape

    # 遍历所有变量计算 reduced cost
    for i in range(m):
        for j in range(n):
            if (i, j) not in basis:
                reduced_cost = C[i, j] - (p_supply[i] - p_demand[j])
                if reduced_cost < 0:
                    return False  
    return True  # 所有 reduced cost >= 0，满足最优条件



In [49]:
import numpy as np
from collections import deque

def find_cycle(basis, entering_variable, m, n):
    """
    Find the cycle (loop) formed after adding the variable (i, j).
    :param basis: The set of current basis variables.
    :param entering_variable: The variable (i, j) entering the basis.
    :param m: Number of supply points.
    :param n: Number of demand points.
    :return: The path of the cycle (in matrix index format).
    """
    # Construct adjacency list
    tree = {i: [] for i in range(m + n)}

    for i, j in basis:
        tree[i].append(j + m)
        tree[j + m].append(i)

    i, j = entering_variable
    tree[i].append(j + m)
    tree[j + m].append(i)

    # Use DFS to detect the cycle
    parent = {}
    cycle = []
    visited = set()

    def dfs(node, prev):
        """DFS recursive function to find the cycle"""
        visited.add(node)
        for neighbor in tree[node]:
            if neighbor == prev:  # Skip the direct backtracking parent node
                continue
            if neighbor in visited:  # Cycle found
                cycle.append((neighbor, node))
                return True
            parent[neighbor] = node
            if dfs(neighbor, node):
                cycle.append((neighbor, node))
                return True
        return False

    # Start searching from the starting node of entering_variable
    start_node = i
    parent[start_node] = None
    dfs(start_node, None)

    # Convert cycle path to matrix index format (i, j)
    matrix_cycle = []
    for u, v in cycle:
        if u < m:  # u is a supply point
            if v >= m:  # v is a demand point
                matrix_cycle.append((u, v - m))
            else:
                matrix_cycle.append((u, v))
        else:  # u is a demand point
            matrix_cycle.append((v, u - m))

    return matrix_cycle




def adjust_flow(X, cycle, entering_variable):
    """
    Adjust the flow based on the identified cycle.
    :param X: The current flow matrix.
    :param cycle: The formed cycle.
    :param entering_variable: The variable entering the basis.
    :return: Adjusted X, the variable leaving the basis.
    """
    min_flow = float('inf')
    leaving_variable = None

    # Determine the minimum flow variable (leaving variable)
    for (i, j) in cycle[1::2]:  # Alternately subtract flow
        if X[i, j] < min_flow:
            min_flow = X[i, j]
            leaving_variable = (i, j)

    # Adjust the flow
    for idx, (i, j) in enumerate(cycle):
        if idx % 2 == 0:
            X[i, j] += min_flow  # Increase flow
        else:
            X[i, j] -= min_flow  # Decrease flow

    return X, leaving_variable





In [50]:
import numpy as np

def initialize_basic_feasible_solution(P, Q):
    """
    Initialize a basic feasible solution using the northwest corner method.
    :param P: Supply vector (m,).
    :param Q: Demand vector (n,).
    :return: Initialized flow matrix X and basis variables.
    """
    m, n = len(P), len(Q)
    X = np.zeros((m, n))  # Initialize the flow matrix.
    basis = set()

    i, j = 0, 0
    supply = P.copy()
    demand = Q.copy()

    while i < m and j < n:
        flow = min(supply[i], demand[j])
        X[i, j] = flow
        basis.add((i, j))

        supply[i] -= flow
        demand[j] -= flow

        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1

    return X, basis

def fix_degenerate_basis(X, basis, m, n):
    """
    Fix degenerate basis to ensure the number of basis variables is m + n - 1.
    :param X: Initial flow matrix.
    :param basis: Initial set of basis variables.
    :param m: Number of supply points.
    :param n: Number of demand points.
    :return: Corrected set of basis variables.
    """
    if len(basis) == m + n - 1:
        return basis  

    # Build a Union-Find structure to check connectivity.
    parent = {i: i for i in range(m + n)}

    def find(x):
        if parent[x] != x:
            parent[x] = find(parent[x])
        return parent[x]

    def union(x, y):
        root_x, root_y = find(x), find(y)
        if root_x != root_y:
            parent[root_x] = root_y

    # Add current basis variables to the Union-Find structure.
    for (i, j) in basis:
        union(i, m + j)

    # Iterate over the matrix to find zero-flow variables as candidates.
    for i in range(m):
        for j in range(n):
            if (i, j) not in basis:
                # Attempt to add a new variable while ensuring no cycle is formed.
                if find(i) != find(m + j):
                    basis.add((i, j))
                    X[i, j] = 0  # Set flow to zero.
                    union(i, m + j)

                    if len(basis) == m + n - 1:  
                        return basis

    return basis  




In [51]:
def network_simplex(C, P, Q, X, basis, y):
    """
    Solve the optimal transport problem using the Network Simplex method.
    :param C: Cost matrix 
    :param P: Supply vector 
    :param Q: Demand vector 
    :param X: Initial flow matrix 
    :param basis: Initial set of basis variables.
    :return: Optimal flow matrix X ,basis ,pivots
    """
    pivot = 0
    n = len(P)
    m = len(Q)

    while True:
        # 1. Compute potential values
        p_supply, p_demand = compute_potentials(basis, C, n, m)
        
        # 2. Record the pivots
        pivot += 1

        # 3. Select the entering variable into the basis
        entering_variable = find_min_reduced_cost(C, p_supply, p_demand, basis, y)

        if entering_variable is None:
            break

        # 4. Find the cycle formed after adding the entering variable
        cycle = find_cycle(basis, entering_variable, n, m)

        # 5. Adjust flow and determine the leaving variable
        X, leaving_variable = adjust_flow(X, cycle, entering_variable)

        # 6. Update basis variables
        basis.remove(leaving_variable)
        basis.add(entering_variable)

    return X, basis, pivot


In [52]:
def select_variables(basis, C, p, q, p_supply, p_demand, A, B):
    """
    Select the subproblem variable set y according to specified rules, 
    and adjust the variable set if all reduced costs are >= 0.
    
    :param basis: Set of basis variables {(i, j)}.
    :param C: Cost matrix.
    :param p: Selection ratio parameter p.
    :param q: Selection ratio parameter q.
    :param p_supply: Potential values of supply points.
    :param p_demand: Potential values of demand points.
    :return: Set y, A, B, count.
    """
    
    total_vars = len(A) + len(B)
    qN = int(q * total_vars)
    pN = int(p * total_vars)
    
    # 1. Remove basis variables from A
    A = [var for var in A if var not in basis]
    
    count = 0  # Number of reduced cost evaluations
    t = 0  # Number of iterations

    while True:
        t += 1
        # 0. If the loop runs too many times, all reduced costs are non-negative, meaning the current solution is optimal.
        if t > total_vars / (0.1 * qN):
            y = None
            return y, A, B, count 
        
        # 1. Select the first qN variables from A and remove them
        selected_A = A[:qN]
        del A[:qN]  
        
        # 2. Select 0.1qN variables from B and remove them
        selected_B = B[:max(1, int(0.1 * qN))]
        del B[:int(0.1 * qN)] 

        # 3. Merge selected_A and selected_B
        selected_vars = selected_A + selected_B

        # 4. Compute reduced costs
        reduced_costs = {}
        for i, j in selected_vars:
            reduced_costs[(i, j)] = C[i, j] - (p_supply[i] - p_demand[j])
            
        count += len(selected_vars)   

        # 5. Check if all reduced costs are >= 0
        if all(cost >= 0 for cost in reduced_costs.values()):
            # If all reduced costs are >= 0, add all variables to B and reselect
            B.extend(selected_vars)
            continue

        # 6. Select the pN variables with the smallest reduced cost as y
        sorted_vars = sorted(reduced_costs.items(), key=lambda x: x[1])
        y = [var[0] for var in sorted_vars[:pN]]

        # 7. Assign remaining variables back to A and B
        for var, cost in sorted_vars[pN:]:
            if cost >= 0:
                B.append(var)
            else:
                A.append(var)

        # 8. Add basis variables to y
        y.extend(basis)
    
        # 9. Add variables in y to A
        A.extend(y)

        return y, A, B, count  



In [53]:
def coordinate_descent_ns(C, P, Q, p, q):
    """
    :param C: Cost matrix (m x n).
    :param P: Supply vector (m,).
    :param Q: Demand vector (n,).
    :param p: Subproblem selection ratio parameter p.
    :param q: Subproblem selection ratio parameter q.
    :return: Optimal flow matrix X, optimal total cost total_cost, pivot, count.
    """
    # Initialization: Compute the initial basic feasible solution using the northwest corner method.
    X, basis = initialize_basic_feasible_solution(P, Q)
    basis = fix_degenerate_basis(X, basis, len(P), len(Q))
    
    # Initialize sets A and B.
    A = [(i, j) for i in range(C.shape[0]) for j in range(C.shape[1])]  
    random.shuffle(A)  
    B = []  
    
    pivot = 0
    count = 0
    N = C.size
    n = C.shape[0]
    m = C.shape[1]
    i = 0
    
    cost_list = []

    while True:
        i += 1
        
        # 0. Recompute the potentials.
        p_supply, p_demand = compute_potentials(basis, C, n, m)
        
        # 1. Select the coordinate set y for the subproblem.
        y, A, B, count_k = select_variables(basis, C, p, q, p_supply, p_demand, A, B)
        if y is None:
            break 

        # 2. Solve the subproblem using the Network Simplex method.
        X, basis, pivot_k = network_simplex(C, P, Q, X, basis, y)
        
        pivot += pivot_k
        count += count_k + pivot_k * p * N
        
        # 3. Compute total cost and record it.
        if i % 10 == 0:
            total_cost = np.sum(C * X)
            cost_list.append(total_cost)
         

    return X, cost_list, count, pivot

